In [47]:
import pandas as pd
import numpy as np
import math
from pulp import *

# x_ij = pd.read_csv("x_ij.csv")
# taus = pd.read_csv("taus.csv")





In [ ]:
# ========= SET-UP =========== #
NUM_AREA = 2
R = 11.5 # Revenue
S = 1.5 # multiplier

# ========== Data ============= #
x_ij = np.matrix([[0.985, 0.615], [0.615, 0.985]]) # [[row]] - x11, x12, x21, x22
c_ij = np.matrix([[0, 4.5], [4.5, 0]])

d_i = [60, 20] # d_1, d_2
n_0i = [10, 40] # n_01, n_02

# ============== LP ============ #
prob = LpProblem("Model", LpMaximize)

# Decision Variable - m_ij
m_ij = np.matrix([[0]*NUM_AREA]*NUM_AREA)
for i in range(NUM_AREA):
    for j in range(NUM_AREA):
        if i!=j:
            m_ij[i,j] = LpVariable("m_%d%d"%(i,j), 0, None, LpInteger)

# Variable - n_1j 
n_1i = []
for i in range(len(n_0i)):
    n_1i.append(LpVariable("n_1%d"%(i+1), 0, None, LpInteger))

# Decision Variable - y_ij
y_ij = np.matrix([[0]*NUM_AREA]*NUM_AREA)
for i in range(NUM_AREA):
    for j in range(NUM_AREA):
        m_ij[i,j] = LpVariable("y_%d%d"%(i,j), 0, None, LpInteger)


# Objective
prob += (sum([S*R*y_ij[i,j]*x_ij[i,j] - m_ij[i,j]*c_ij[i,j] for i in range(NUM_AREA) for j in range(NUM_AREA)])) 

# Subject To
# n_1i
for i in range(len(NUM_AREA)):
    prob += n_1i[i] == n_0i[i] + 
prob += n11 == n01 + m21 - m12 
prob += n12 == n02 + m12 - m21
prob += n01 >= m12
prob += n02 >= m21
prob += y11 + y12 <= d1
prob += y21 + y22 <= d2
prob += y11*x11 + y21*x21 <= n11
prob += y21*x21 + y22*x22 <= n12



y = [y11, y12, y21, y22]
x = [x11, x12, x21, x22]
c = [c11, c12, c21, c22]
m = [0, m12, m21, 0]




In [48]:
# Old Model

x11, x12, x21, x22 = 0.985, 0.615, 0.615,0.985
c11, c12, c21, c22 = 0, 4.5, 4.5, 0
d1, d2 = 60,20
n01, n02 = 10, 40


prob = LpProblem("Model", LpMaximize)
m12 = LpVariable("m12", 0, None, LpInteger)
m21 = LpVariable("m21", 0, None, LpInteger)

n11 = LpVariable("n11", 0, None, LpInteger)
n12 = LpVariable("n12", 0, None, LpInteger)

y12 = LpVariable("y12", 0, None, LpInteger)
y21 = LpVariable("y21", 0, None, LpInteger)
y11 = LpVariable("y11", 0, None, LpInteger)
y22 = LpVariable("y22", 0, None, LpInteger)

# Constants
R = 11.5
s = 1.5 # multiplier
y = [y11, y12, y21, y22]
x = [x11, x12, x21, x22]
c = [c11, c12, c21, c22]
m = [0, m12, m21, 0]

prob += (sum([s*R*y[i]*x[i]  - m[i]*c[i] for i in range(len(y))])) 
prob += n11 == n01 + m21 - m12 
prob += n12 == n02 + m12 - m21
prob += n01 >= m12
prob += n02 >= m21
prob += y11 + y12 <= d1
prob += y21 + y22 <= d2
prob += y11*x11 + y21*x21 <= n11
prob += y21*x21 + y22*x22 <= n12



In [49]:
prob.solve()
value(prob.objective)
for v in prob.variables():
    print (v.name, "=", v.varValue)

m12 = 0.0
m21 = 20.0
n11 = 30.0
n12 = 20.0
y11 = 30.0
y12 = 30.0
y21 = 0.0
y22 = 20.0


603.0

In [52]:
l = [LpVariable("m1", 0, None, LpInteger), LpVariable("m2", 0, None, LpInteger)]

In [53]:
for v in l:
    print (v.name, "=", v.varValue)

m1 = None
m2 = None


In [54]:
l = [[2,3,4], [1,3,5], [3,4,5]]
[l[i][j] for i in range(len(l)) for j in range(len(l))]

[2, 3, 4, 1, 3, 5, 3, 4, 5]

In [65]:
x = np.matrix([[0]*3]*3)
x[0,1] = 1
x

matrix([[0, 1, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [67]:
x = np.matrix([[2,3],[1,4]])
sum(x[i,0] for i in range(2))

3